# Self-translate

英語以外の言語のプロンプトでは言語モデルはその能力をフルに活用できていない，という研究に関する論文．self-translateは入力を一度英語に翻訳して出力を得てから元の言語に戻す手法．
- 論文:https://arxiv.org/abs/2308.01223

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/nlp/llm_prompt_engineering/self_translate.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from langchain.llms import OpenAI

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

llm_model = OpenAI(model_name= 'gpt-3.5-turbo', temperature=0)

/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### 通常のプロンプト(Few-shot prompting)

In [11]:
prompt = """
アンディはゼラニウムを90本植え、ペチュニアをゼラニウムより40本少なく植える全部で何本の花を植えたでしょう？
"""

In [12]:
output = llm_model(prompt)
print(output)

アンディはゼラニウムを90本植えたので、ゼラニウムの本数は90本です。
ペチュニアの本数はゼラニウムより40本少なく植えたので、ゼラニウムの本数-40本です。
したがって、花の総数は90本 + (90本 - 40本) = 90本 + 50本 = 140本です。


### Self-translate

In [5]:
# 英語に翻訳する
prompt1 = """
Japanese:メアリは緑の魔女を叩かなかった．
English:Mary did not slap the green witch.

Japanese:アンディはゼラニウムを90本植え、ペチュニアをゼラニウムより40本少なく植える。
全部で何本の花を植えたでしょう？
English:
"""


In [6]:
output1 = llm_model(prompt1)
print(output1)

Andy planted 90 geraniums and planted 40 fewer petunias than geraniums.
How many flowers did he plant in total?


In [7]:
# 問題を解く
prompt2 = """
Question: John had 12 apples and ate a quarter of them. How many apples does he have now?
Stepwise solution: John ate 12÷4=3 apples. Therefore, he has 12-3=9 apples now. The answer is 9.

Question: {output1_}
Stepwise solution:
""".format(output1_ = output1)

In [8]:
output2 = llm_model(prompt2)
print(output2)

Andy planted 90-40=<<90-40=50>>50 petunias.
Therefore, he planted a total of 90+50=<<90+50=140>>140 flowers.


In [9]:
# 元の言語に直す
prompt3 = """
English:Mary did not slap the green witch.
Japanese:メアリは緑の魔女を叩かなかった．

English:{output2_}
Japanese:
""".format(output2_ = output2)

In [10]:
output3 = llm_model(prompt3)
print(output3)

アンディは90-40=50のペチュニアを植えました。
したがって、彼は合計で90+50=140の花を植えました。
